In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/hkabbasi6/kaggle.git 

In [ ]:
!wget https://raw.githubusercontent.com/hkabbasi6/kaggle/main/helper_functions.py

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf


In [ ]:
from helper_functions import make_image_database_from_folder


In [ ]:

sub = make_image_database_from_folder(train_path="/kaggle/input/fundus-pytorch/train",
                                      valid_path="/kaggle/input/fundus-pytorch/val",
                                      test_path="/kaggle/input/fundus-pytorch/test",
                                      chart_figure=(4,4),before_process=False,
#                                     #train_percent=0.1,valid_percent=0.1,
                                      valid_balance=True,
                                      train_balance=True)

In [ ]:
class_name = sub[0]
train_data = sub[1]
valid_data = sub[2]
test_image = sub[3]
test_label = sub[4]


In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy(policy="mixed_float16") # set global policy to mixed precision

mixed_precision.global_policy() # should output "mixed_float16" (if your GPU is compatible with mixed precision)

In [ ]:
# make model

import tensorflow as tf


tf.random.set_seed(42)

model = tf.keras.models.Sequential([
    # tf.keras.layers.RandmomFlip('horizontal', ),
    # tf.keras.layers.RandomRotation(0.2),
    # tf.keras.layers.RandomBrightness(0.2),
    # tf.keras.layers.RandomContrast(0.2),
    # tf.keras.layers.RandomZoom(0.2),
    # tf.keras.layers.RandomTranslation(0.2, 0.2),
    # tf.keras.layers.RandomWidth(0.2),
    # tf.keras.layers.RandomHeight(0.2),
    # tf.keras.layers.RandomCrop
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1, activation='sigmoid',dtype=tf.float32),

])

# compile model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

# fit model


In [ ]:
# Check the dtype_policy attributes of layers in our model
for layer in model.layers:
    print(layer.name, layer.trainable, layer.dtype, layer.dtype_policy) # Check the dtype policy of layers

In [ ]:
SACE_MODEL_WEIGHTS = True

if SACE_MODEL_WEIGHTS:
  # weights_path = "drive/MyDrive/Machine learning data/log/test model weight/Glucose/glucose_{val_accuracy:.4f}"
  weights_path = "Glucose/glucose"

  checkpoint = tf.keras.callbacks.ModelCheckpoint(
    weights_path,
    monitor='val_accuracy',  # Monitor the validation loss
    save_weights_only=True,  # Save only the best model
    save_best_only=True,  # Save only the best model
    mode='max',  # Save the model with the minimum validation loss
    save_freq='epoch',  # Save the model after each epoch
    verbose=1,  # Print a message when saving the model
)


else:
  filepath = "log/test model/glucose_{val_accuracy:.4f}.h5"

  # Define the callback to save the best model
  checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath,
    monitor='val_accuracy',  # Monitor the validation loss
    save_best_only=True,  # Save only the best model
    mode='max',  # Save the model with the minimum validation loss
    save_freq='epoch',  # Save the model after each epoch
    verbose=1,  # Print a message when saving the model
   )



In [ ]:

history = model.fit(train_data,
                    #  steps_per_epoch=len(train_data)*0.1,
            epochs=2000,
                    verbose=0,
            validation_data=valid_data,
                    #  validation_steps=len(valid_data)*0.1,
            validation_freq=1, # check validation metrics every epoch
                     callbacks=[
                         tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=40,restore_best_weights=True),
                         tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.9, patience=10, verbose=1, min_lr=0.00000001),
                         checkpoint
                     ]

            )

In [ ]:
if SACE_MODEL_WEIGHTS:
  # check for latest file in folder
  path = tf.train.latest_checkpoint("Glucose")
  # path = weights_path
  print(path)

  # lod weights
  model.load_weights(path)

  print("Loaded weights from", path)

  # evaluate the model
  model_result = model.evaluate(test_image, test_label)

  # save model
  model.save(f"log/test model/Glucose_100_{model_result[1]:.4f}.h5")
  print(f"Saving model at {model_result[1]:.4f}.h5")


In [ ]:
from helper_functions import binary_plot_loss_history_confiusion_matrix

binary_plot_loss_history_confiusion_matrix(model, pl_history=history,pl_X_test=test_image,pl_y_test=test_label,classes=["No Diabetes","Diabetes"])